In [1]:
import tkinter as tk
from openai import OpenAI
import os


# Set your OpenAI API key here
OPENAI_API_KEY = 'Paste api key here'

client = OpenAI(api_key=OPENAI_API_KEY)

# Function to format messages for the API
def format_message(role, content):
    return {"role": role, "content": content}

# Function to send the message and get the response using the OpenAI API
def get_response(messages):
    completion = client.chat.completions.create(
        model='gpt-4-1106-preview',  # Latest model
        messages=messages,
    )
    content = completion.choices[0].message.content
    return content

# Function to summarize text using the OpenAI GPT API (chat model)
def summarize_text(text):
    try:
        # Prepare the instruction with the text for summarization
        instructions = """
        First, identify if the following text is a message to the user. If the text is a message to the user, summarize it in 
        the fewest possible sentences such that the summary only includes the main point of the message, and 
        is at least half the length of the original message. Then list three possible responses to it based on the         
        details of the message. The possible responses should be formatted such that it just says 
        "Possible Responses" and beneath it the repsonses ordered by number and spaced out for easy viewing.
        If the text is not in the form of a message to the user , simply summarize the following text in the least number of 
        possible sentences to get the point across, and do not list possible responses. If the text is in a different language, make sure it is translated 
        into English.
        
        Text to summarize: 
        """ + text
        message = format_message("system", instructions)
        messages = [message]

        # Get the summary from the OpenAI API
        summary = get_response(messages)
        return summary
    except Exception as e:
        return f"Error: {str(e)}"

# Function to dynamically create response buttons if "Possible Responses" are detected
def create_response_buttons(response_text):
    if "Possible Responses" in response_text:
        # Extract possible responses (assuming they are in the form: "1. Response one", "2. Response two", etc.)
        responses = [line.strip() for line in response_text.split("\n") if line.strip().startswith(("1.", "2.", "3."))]

        # Create the buttons if there are exactly 3 possible responses
        if len(responses) == 3:
            # Clear any existing buttons before adding new ones
            for widget in response_button_frame.winfo_children():
                widget.destroy()

            for i, response in enumerate(responses):
                button = tk.Button(response_button_frame, text=f"Send Response {i+1}", command=lambda r=response: print(f"Sending: {r}"))
                button.pack(pady=5)  # Add padding for vertical spacing

# Create the main window
root = tk.Tk()
root.title("NRT: Not Reading That")
root.geometry("400x600")

# Create a Text widget for the input text box with reduced height
text_box = tk.Text(root, wrap='word', height=10)
text_box.pack(expand=False, fill='both', padx=10, pady=10)

# Function for the "I am not reading that" button
def button_action():
    # Get the content from the text_box
    input_text = text_box.get(1.0, tk.END).strip()

    # Use the summarize_text function to get the summary from OpenAI API
    if input_text:
        summary = summarize_text(input_text)

        # Update the output_box with the summary
        output_box.config(state=tk.NORMAL)  # Enable writing to the output box
        output_box.delete(1.0, tk.END)  # Clear the current contents
        output_box.insert(tk.END, summary)  # Insert the summary
        output_box.config(state=tk.DISABLED)  # Disable the output box to make it read-only

        # Check if the response contains "Possible Responses"
        create_response_buttons(summary)

button = tk.Button(root, text="I am not reading that", command=button_action)
button.pack(pady=10)

# Create a second text box for display, make it read-only
output_box = tk.Text(root, wrap='word', height=10)
output_box.pack(expand=False, fill='both', padx=10, pady=10)
output_box.config(state=tk.DISABLED)  # Set the text box to read-only initially

# Define the function for the Save button
def save_content():
    # Get the content of the output_box
    gpt_output = output_box.get(1.0, tk.END).strip()

    if gpt_output:  # Only save if there is content
        file_path = "NRT_Logs.txt"  # File name to save the logs
        
        # Open file in append mode, if it doesn't exist, it will create a new one
        with open(file_path, "a") as file:
            file.write(f"GPT Output:\n{gpt_output}\n{'='*40}\n")
        print(f"Content saved to {file_path}")

# Define the function for the Logs button to open the log file
def open_logs():
    file_path = "NRT_Logs.txt"
    
    # Check if the file exists before trying to open it
    if os.path.exists(file_path):
        os.startfile(file_path)  # This will open the file with the default application (Notepad on Windows)
        print(f"Opening {file_path}")
    else:
        print(f"{file_path} does not exist.")

# Create a frame for Save and Logs buttons
button_frame = tk.Frame(root)
button_frame.pack(pady=10)

save_button = tk.Button(button_frame, text="Save", command=save_content)
save_button.pack(side='left', padx=10)

logs_button = tk.Button(button_frame, text="Logs", command=open_logs)
logs_button.pack(side='left', padx=10)

# Create a separate frame for the response buttons to appear below Save and Logs
response_button_frame = tk.Frame(root)
response_button_frame.pack(pady=10)

# Run the Tkinter event loop
root.mainloop()

Sending: 1. Haha, thanks for the heads up, Mom! I think my nose is fine as it is though.
Sending: 3. Don't worry, Mom, I haven’t used chia seeds since that smoothie incident. All good here!
